In [2]:
import numpy as np
from lib.defaults import Main
import json
from lib.controller import dynamic, decompose
from lib.lqr import classical_lqr
from lib.simple import Sim
from scipy.linalg import solve_lyapunov, solve

In [3]:
# params I need 
main = Main()
a = main.a
r2 = main.r2_dynamic
pmd_slice = main.pmd_slice
taus = main.taus
tau = main.tau
# prms
n = main.n
n_e = main.n_e

with open('setup_prms.json') as f:
    prms = json.load(f)
c = np.array(prms['c'])

In [4]:
tau_z, tau_y = main.taus
q = Sim().q_mov(c)

# I pick out codes from controller.dynamic

In [5]:
n, _ = a.shape
n_pmd = len(pmd_slice)
n_i = n - n_e
tau_z, tau_y = taus
const_y = (tau / tau_y) * np.eye(n_e)
const_z = (tau / tau_z) * np.eye(n_e)

yx = 0.1
x = np.random.binomial(1, yx, size=(n_e, n_e)) / (yx * n_e)
yx = np.hstack((x, np.zeros((n_e, n_i))))

a = np.block([
    [a, np.zeros((n, n_e)), np.zeros((n, n_e))],
    [(tau / tau_y) * yx, -const_y, np.zeros((n_e, n_e))],
    [np.zeros((n_e, n)), const_z, -const_z]
])

b = np.block([
    [np.eye(n)],
    [np.zeros((n_e + n_e, n))]
])

c = np.hstack((np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.eye(n_e)))

q = np.block([
    [q, np.zeros((n, n_e)), np.zeros((n, n_e))],
    [np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.zeros((n_e, n_e))],
    [np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.zeros((n_e, n_e))]
])

In [69]:
def adam_optimizer(params, gradients, lr=3, beta1=0.9, beta2=0.999, eps=1e-8, t=0):
    """
    Adam optimizer implementation in NumPy.
    Args:
        params (list): List of NumPy arrays representing the trainable parameters.
        gradients (list): List of NumPy arrays representing the gradients of the trainable parameters.
        lr (float): Learning rate. 1e-0
        beta1 (float): Exponential decay rate for the first moment estimates.
        beta2 (float): Exponential decay rate for the second moment estimates.
        eps (float): Small constant to avoid division by zero.
        t (int): Current iteration number (starting from 0).
    Returns:
        params (list): Updated parameters after applying Adam optimizer.
    """
    # Initialize the first and second moment estimates
    m = [np.zeros_like(p) for p in params]
    v = [np.zeros_like(p) for p in params]
    
    # Update iteration number
    t += 1
    
    # Compute the first and second moment estimates
    for i in range(len(params)):
        m[i] = beta1 * m[i] + (1 - beta1) * gradients[i]
        v[i] = beta2 * v[i] + (1 - beta2) * gradients[i] ** 2
    
    # Bias correction for first and second moment estimates
    m_hat = [m[i] / (1 - beta1 ** t) for i in range(len(params))]
    v_hat = [v[i] / (1 - beta2 ** t) for i in range(len(params))]
    
    # Update parameters
    for i in range(len(params)):
        params[i] -= lr * m_hat[i] / (np.sqrt(v_hat[i]) + eps)
    
    return params

# I pick out codes from lqr.output_lqr

In [78]:
n, _ = a.shape
iden = np.eye(n)
clo, raw = (n_pmd, n_e)
npara = clo * raw
bt = b.T
ct = c.T

def k_of(z):
    return z @ c

def s_of(acl):
    return solve_lyapunov(acl, -iden)

def p_of(acl, k):
    return solve_lyapunov(acl.T, -(q + (r2 * k.T @ bt @ b @ k)))

def grad_z(k, p, s):
    return 2 * bt @ (p + (r2 * b @ k)) @ s @ ct   


def f_df(z):
    z = np.reshape(z, (clo, raw))
    k = k_of(z)
    acl = a + (b @ k) 
    p = p_of(acl, k)  
    return np.trace(p)

def grad_func(z):
    z = np.reshape(z, (clo, raw))
    k = k_of(z)
    acl = a + (b @ k) 
    s = s_of(acl)
    p = p_of(acl, k)
    g_ = grad_z(k, p, s)    
    return g_.reshape(npara)


z0 = classical_lqr(r2, q, a, b) @ solve(c @ c.T, c).T
z0 = z0.reshape(npara)

n_iter = 38
for i in range(n_iter):                        
    l = f_df(z0)
    z0 = np.array(adam_optimizer([z0], grad_func(z0))[0])
    print(f'iteration {i+1}/{n_iter} | train loss: {l:.6f}')
#     if l < 1e-6:
#         print(f'train loss: {l:.6f}')
#         break


iteration 1/38 | train loss: 503.293667
iteration 2/38 | train loss: 534.703156
iteration 3/38 | train loss: 521.196449
iteration 4/38 | train loss: 506.533778
iteration 5/38 | train loss: 491.591272
iteration 6/38 | train loss: 476.537924
iteration 7/38 | train loss: 461.429420
iteration 8/38 | train loss: 446.289490
iteration 9/38 | train loss: 431.129957
iteration 10/38 | train loss: 415.957373
iteration 11/38 | train loss: 400.775662
iteration 12/38 | train loss: 385.587319
iteration 13/38 | train loss: 370.394006
iteration 14/38 | train loss: 355.196870
iteration 15/38 | train loss: 339.996733
iteration 16/38 | train loss: 324.794196
iteration 17/38 | train loss: 309.589710
iteration 18/38 | train loss: 294.383618
iteration 19/38 | train loss: 279.176189
iteration 20/38 | train loss: 263.967634
iteration 21/38 | train loss: 248.758123
iteration 22/38 | train loss: 233.547791
iteration 23/38 | train loss: 218.336752
iteration 24/38 | train loss: 203.125096
iteration 25/38 | train l

# The loss function is computed negative ↑

In [63]:
from scipy.optimize import rosen, rosen_der

In [67]:
def adam_optimizer(params, gradients, lr=1e-1, beta1=0.9, beta2=0.999, eps=1e-8, t=0):
    """
    Adam optimizer implementation in NumPy.
    Args:
        params (list): List of NumPy arrays representing the trainable parameters.
        gradients (list): List of NumPy arrays representing the gradients of the trainable parameters.
        lr (float): Learning rate. 1e-0
        beta1 (float): Exponential decay rate for the first moment estimates.
        beta2 (float): Exponential decay rate for the second moment estimates.
        eps (float): Small constant to avoid division by zero.
        t (int): Current iteration number (starting from 0).
    Returns:
        params (list): Updated parameters after applying Adam optimizer.
    """
    # Initialize the first and second moment estimates
    m = [np.zeros_like(p) for p in params]
    v = [np.zeros_like(p) for p in params]
    
    # Update iteration number
    t += 1
    
    # Compute the first and second moment estimates
    for i in range(len(params)):
        m[i] = beta1 * m[i] + (1 - beta1) * gradients[i]
        v[i] = beta2 * v[i] + (1 - beta2) * gradients[i] ** 2
    
    # Bias correction for first and second moment estimates
    m_hat = [m[i] / (1 - beta1 ** t) for i in range(len(params))]
    v_hat = [v[i] / (1 - beta2 ** t) for i in range(len(params))]
    
    # Update parameters
    for i in range(len(params)):
        params[i] -= lr * m_hat[i] / (np.sqrt(v_hat[i]) + eps)
    
    return params

In [68]:
z0 = np.zeros(200)
n_iter = 100
for i in range(n_iter):                        
    l = rosen(z0)
    z0 = np.array(adam_optimizer([z0], rosen_der(z0))[0])
    print(f'iteration {i+1}/{n_iter} | train loss: {l:.6f}')
    if l < 1e-6:
        print(f'train loss: {l:.6f}')
        break

iteration 1/100 | train loss: 199.000000
iteration 2/100 | train loss: 322.379999
iteration 3/100 | train loss: 636.799998
iteration 4/100 | train loss: 975.099998
iteration 5/100 | train loss: 1217.879999
iteration 6/100 | train loss: 1293.500000
iteration 7/100 | train loss: 1178.080002
iteration 8/100 | train loss: 895.500003
iteration 9/100 | train loss: 517.400003
iteration 10/100 | train loss: 163.180003
iteration 11/100 | train loss: 0.000000
train loss: 0.000000
